In [2]:
# Studying Aux GTFS files to support in the cleaning
# Spark Config
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# spark = sparkSession
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

# installing necessary packages for notebook session
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# _c0 = line number in GTFS system
# _c1 = complement of the line
# _c2 = line_id in the MO file
# _c3 = direction

print("total","unique_line_id","unique_line_number_complement")
for day in range(1,32):
    routes = spark.read.csv("s3://mobility-traces-sp/aux-files/route-files/AL_1510"+ str(day) + ".txt",header=False)
    
    # checking if a line of MO file can have more than one route in AL file (different directions)
    checking_line_id_duplicate = routes.drop_duplicates(subset=["_c2"])
    
    # Checking it there are same line_numer+complement belonging to omre than one line_id from MO file
    checking_line_number_duplicate = routes.drop_duplicates(subset=["_c0","_c1"])
    
    print(routes.count(),checking_line_id_duplicate.count(),checking_line_number_duplicate.count())
    
    # checking unique values for direction in AL file (1 or 2)
    print(routes.select("_c3").distinct().collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

total unique_line_id unique_line_number_complement
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2782 2782 1391
[Row(_c3='1'), Row(_c3='2')]
2784 2784 1392
[Row(_c3='1'), Row(_c3='2')]
2784 2784 1392
[Row(_c3='1'), Row(_c3='2')]
2784 2784 1392
[Row(_c3='1'), Row(_c3='2')]
2784 2784 1392
[Row(_c3='1'), Row(_c3='2')]
2784 2784 1392
[Row(_c3='1'), Row(_c3='2')]
2784 2784 1392
[Row(_c3='1'), Row(_c3='2')]
2784 2784 1392
[Row(_c3='1'), Row(_c3='2')]
2784 2784 1392
[Row(_c3='

In [4]:
# Results
# There are no cases where the line_id from MO file has more than route_id in AL_file
# There are# In the GFTS the direction is represented by 1 or 0, 
# so you have to find out which one (1 or 2) represents the direction 1 and the direction 0 more than one line_id for a line_number+complement
# The options to indicate direction are 1 and 2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# In the GFTS the direction is represented by 1 or 0, 
# so you have to find out which one (1 or 2) represents the direction 1 and the direction 0

In [8]:
from pyspark.sql.types import *

AL_schema = StructType([
    StructField("line_number", StringType()),
    StructField("complement", StringType()),
    StructField("line_id", IntegerType()),
    StructField("direction", IntegerType())
])

# reading bus records of october 1 2015
traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/6-records-inside-sp-in-october-with-time-variation/MO_15101/")
AL_df = spark.read.csv("s3://mobility-traces-sp/aux-files/route-files/AL_15101.txt",header ='false',schema=AL_schema)

# select distinct line_ids in MO file
line_id_distinct = traces.select('line_id').distinct()

import random

# list of distinct line_id in MO file
lines = list(line_id_distinct.collect())

random_num = random.randint(0,len(lines))

# selecting random line_id
line_id = lines[random_num]["line_id"]

# finding the liner_number for line_id

# error no treated --> line_id can not have a AL, for prod code, this error must be treated
line_number_row = AL_df.filter("line_id == " +  str(line_id)).collect()

print(line_number_row)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(line_number='2435', complement='10', line_id=210, direction=1)]

In [9]:
line_number = line_number_row[0]["line_number"]
line_complement = line_number_row[0]["complement"]
line_direction = line_number_row[0]["direction"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# In the GTFS files the direction is represented by 1 and 0, and in the AL file the direction can be 1 or 2
# so we need to identify the right ones
# Identifying direction 1= 0 or 1= 1 and 2 = 1 or 2 =0?

In [11]:
# The random selection was [Row(line_number='2435', complement='10', line_id=210, direction=1)]

# reading trips.txt (csv)
trips = spark.read.csv("s3://mobility-traces-sp/aux-files/gtfs/trips.csv",header=True)

# possible trips - direction 1 and 0
trip_id_0 = f'{line_number}-{line_complement}-0'
trip_id_1 = f'{line_number}-{line_complement}-1'
print(trip_id_0,trip_id_1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2435-10-0 2435-10-1

In [12]:
trips.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+---------+--------------------+------------+--------+
|route_id|service_id|  trip_id|       trip_headsign|direction_id|shape_id|
+--------+----------+---------+--------------------+------------+--------+
| 1015-10|       USD|1015-10-0|Chácara Maria Tri...|           0|   61150|
| 1016-10|       USD|1016-10-0|        Center Norte|           0|   53755|
| 1016-10|       USD|1016-10-1|  Cemiterio Do Horto|           1|   53756|
| 1017-10|       USD|1017-10-0|  Conexão Vila Iorio|           0|   61079|
| 1017-10|       USD|1017-10-1|               Perus|           1|   61080|
| 1018-10|       USD|1018-10-0|       Metrô Santana|           0|   52197|
| 1018-10|       USD|1018-10-1|           Vila Rosa|           1|   52198|
| 1021-10|       US_|1021-10-0|   Terminal Pirituba|           0|   58785|
| 1021-10|       US_|1021-10-1|   Cohab Brasilândia|           1|   58786|
| 1024-10|       USD|1024-10-0|Conexão Petrônio ...|           0|   58765|
| 1024-10|       USD|1024

In [14]:
# remember to treat the shape can not exists

# Selecting shapes for the trips
shape_id_0 = trips.filter("trip_id == '" + trip_id_0+ "'").collect()[0]["shape_id"]
shape_id_1 = trips.filter("trip_id == '" + trip_id_1+ "'").collect()[0]["shape_id"]
print(shape_id_0,shape_id_1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

58695 53709

In [16]:
# reading shapes from GTFS file collection
shapes = spark.read.csv("s3a://mobility-traces-sp/aux-files/gtfs/shapes.csv",header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# selecting the points of the shapes
shape_0 = shapes.filter("shape_id == '" + shape_id_0+ "'")
shape_1 = shapes.filter("shape_id == '" + shape_id_1+ "'")

shape_0.repartition(1).write.csv("s3://mobility-traces-sp/tests-map-views/1-shapes-direction-AL-file/day_15101_shape_"+shape_id_0+"/")
shape_1.repartition(1).write.csv("s3://mobility-traces-sp/tests-map-views/1-shapes-direction-AL-file/day_15101_shape_"+shape_id_1+"/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# selecting and saving traces with the line_id
traces_selected = traces.filter("line_id == "+ str(line_id))
traces_selected.repartition(1).write.csv("s3://mobility-traces-sp/tests-map-views/1-shapes-direction-AL-file/day_15101_line_"+str(line_id)+"/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# https://towardsdatascience.com/mapping-geograph-data-in-python-610a963d2d7f

In [38]:
# # Using folium to plot map
# import folium

# map_folium = folium.Map([-23.550164466,-46.633664132], zoom_start=5)

# tile = folium.TileLayer('cartodbpositron').add_to(map_folium)

# # Plotting shapes 53709
# for coord in coords_shape_53709:
#     icon = folium.features.CustomIcon('/home/hadoop/green-point.png', icon_size=(10,10))
#     folium.Marker(coord,icon=icon).add_to(map_folium)

# # Plotting shapes 53709
# for coord in coords_shape_58695:
#     icon = folium.features.CustomIcon('/home/hadoop/blue-point.png', icon_size=(10,10))
#     folium.Marker(coord,icon=icon).add_to(map_folium)
    
# # Plotting traces from line 210
# for coord in coords_trace_line_210:
#     icon = folium.features.CustomIcon('/home/hadoop/red-point.png', icon_size=(10,10))
#     folium.Marker(coord,icon=icon).add_to(map_folium)

    
# map_folium.save("/tmp/testing-shapes-AL-direction.html")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…